In [9]:
import csv

In [46]:
class LVQNeuron:
    def __init__(self, name):
        self.name = name
        self.weights = []
    
    def setWeights(self, weights):
        for weight in weights:
            self.weights.append(weight)
        
    def __str__(self):
        return "Hi I'm " + self.name
    
    def __len__(self):
        return len(self.weights)

In [47]:
class LVQNet:
    def __init__(self, inCount, outCount):
        self.inputs  = inCount
        self.outputs = outCount
        self.neurons = {} # numbered index map (to outputs)
    
    def assignNeuron(self, index, neuron):
        self.neurons[index] = neuron
    
    def __str__(self):
        return str(self.neurons)
    
    def __len__(self):
        return len(self.neurons)
    
    #TODO Port LVQ Constructor to LVQNet
    def construct(self, inputs, outputs)

In [48]:
class LVQConstructor:
    def __init__(self, inputs, outputs):
        self.network = LVQNet(inputs, outputs)
        temp_name = ""
        
        for n in range(outputs):
            temp_name += "." # purely aesthetic
            curr_neuron = LVQNeuron(temp_name)
            curr_neuron.setWeights([1,1,1,11,1,1,1,1,1,11,11,1,1,11,1,1,1,1,1,1,1])
            self.network.assignNeuron(n, curr_neuron)
            
    def __str__(self):
        return str(self.network)

In [49]:
if __name__ == '__main__':
    archie = LVQConstructor(1025, 3)
    print archie
    

{0: <__main__.LVQNeuron instance at 0x10cb0bb90>, 1: <__main__.LVQNeuron instance at 0x10cb0bc20>, 2: <__main__.LVQNeuron instance at 0x10cb0bc68>}
